<a href="https://colab.research.google.com/github/hethatgithub/fashion-mnist-app/blob/main/ML_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import warnings
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Suppress warnings
warnings.filterwarnings("ignore")

# Define directories for dataset
base_dir = 'Dataset/'  # Update if needed
MildDemented_Dir = os.path.join(base_dir, 'Mild_Demented')
ModerateDemented_Dir = os.path.join(base_dir, 'Moderate_Demented')
NonDemented_Dir = os.path.join(base_dir, 'Non_Demented')
VeryMildDemented_Dir = os.path.join(base_dir, 'Very_Mild_Demented')

# Prepare dataset
filepaths = []
labels = []
dict_list = [MildDemented_Dir, ModerateDemented_Dir, NonDemented_Dir, VeryMildDemented_Dir]
class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']

for i, j in enumerate(dict_list):
    file_list = os.listdir(j)
    for f in file_list:
        filepath = os.path.join(j, f)
        filepaths.append(filepath)
        labels.append(class_labels[i])

FileSeries = pd.Series(filepaths, name='filepaths')
LabelSeries = pd.Series(labels, name='labels')
Alzheimer_df = pd.concat([FileSeries, LabelSeries], axis=1)

# Print class distribution
print(Alzheimer_df["labels"].value_counts())

# Data Preprocessing
# Set image size
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

# Split dataset into training and validation
train_df, valid_df = train_test_split(Alzheimer_df, test_size=0.2, stratify=Alzheimer_df['labels'], random_state=42)

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

# Model Creation
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze base model

# Create a new model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=20,
                    callbacks=[early_stop])

# Save the trained model
model.save('alzheimer_detection_model.h5')

# Gradio Interface
def predict(image_path):
    img = image.load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']
    return class_labels[class_idx]

interface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.Image(type="filepath"),
    outputs="text",
    title="Alzheimer's Disease Detection",
    description="Upload an MRI image to detect Alzheimer's disease."
)

# Launch the Gradio app
interface.launch()

labels
Non Demented          471
Mild Demented         400
Very Mild Demented    282
Moderate Demented      64
Name: count, dtype: int64
Found 973 validated image filenames belonging to 4 classes.
Found 244 validated image filenames belonging to 4 classes.
Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 274s 9s/step - accuracy: 0.3529 - loss: 2.5537 - val_accuracy: 0.4795 - val_loss: 1.0638
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 270s 9s/step - accuracy: 0.4746 - loss: 1.1320 - val_accuracy: 0.5533 - val_loss: 1.0261
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 323s 9s/step - accuracy: 0.4740 - loss: 1.1030 - val_accuracy: 0.5738 - val_loss: 0.9721
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 322s 9s/step - accuracy: 0.5405 - loss: 1.0077 - val_accuracy: 0.5533 - val_loss: 0.9588
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 270s 9s/step - accuracy: 0.5257 - loss: 1.0190 - val_accuracy: 0.5615 - val_loss: 0.9553
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 353s 10s/step - accuracy: 0.5087 - loss: 1.0021 - val_accuracy: 0.5615

AttributeError: module 'gradio' has no attribute 'inputs'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define a simple CNN model (adjust as needed)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),  # Adjust input shape
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # Assuming you have 4 classes
])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Match the input shape of your model
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Match the input shape of your model
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
def predict(image_path):
    image = load_and_preprocess_image(image_path)  # Load and preprocess the image
    print(f"Input shape: {image.shape}")  # Debugging line
    predictions = model.predict(image)  # Make predictions
    class_index = np.argmax(predictions)  # Get the index of the highest prediction
    return class_labels[class_index]  # Return the predicted class

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
# After training your model
model.save('alzheimer_detection_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('alzheimer_detection_model.h5')

In [ ]:
import os
import numpy as np
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing import image

# Load your pre-trained model
model = load_model('alzheimer_detection_model.h5')

# Compile the model (if needed, based on your original training configuration)
optimizer = Adam(learning_rate=1e-3)  # Adjust the learning rate as necessary
loss_function = SparseCategoricalCrossentropy()  # Adjust based on your original loss function
metrics = ['accuracy']  # Add any additional metrics you were using
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Define your class labels (adjust according to your dataset)
class_labels = ['Non Demented', 'Very Mild Demented', 'Mild Demented', 'Moderate Demented']

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Adjust the target size based on your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

# Your prediction function
def predict(image_path):
    # Load and preprocess the image
    image = load_and_preprocess_image(image_path)
    # Make predictions
    predictions = model.predict(image)
    # Print raw prediction probabilities for debugging
    print("Raw Predictions:", predictions)
    # Interpret predictions
    class_index = np.argmax(predictions)
    return class_labels[class_index]

# Setup the Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="filepath"),  # Use gr.Image for newer versions
    outputs="text",
    title="Alzheimer's Disease Detection",
    description="Upload an MRI image to detect Alzheimer's disease."
)

# Launch the interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://273fd2aa9c832dbce9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load your pre-trained binary classification model
model = load_model('alzheimer_detection_model.h5')

# Define your image preprocessing function
def load_and_preprocess_image(image_path):
    try:
        # Load the image, resize, and normalize it
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))  # Change target_size as needed
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image /= 255.0  # Normalize to [0, 1]
        return image
    except Exception as e:
        return f"Error loading image: {e}"

# Your prediction function
def predict(image_path):
    try:
        # Load and preprocess the image
        image = load_and_preprocess_image(image_path)
        if isinstance(image, str):  # Check if the error message was returned
            return image

        # Make predictions
        predictions = model.predict(image)

        # Print the raw predictions for debugging
        print("Raw predictions:", predictions)

        # Interpret predictions
        class_index = np.argmax(predictions, axis=-1)  # Get index of the highest probability
        print("Class index:", class_index)

        if predictions[0][0] >= 0.2:  # If you have a sigmoid output
            return "Alzheimer's Disease Detected"
        else:
            return "Not Detected"
    except Exception as e:
        return f"Prediction error: {e}"

# Setup the Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title="Alzheimer's Disease Detection",
    description="Upload an MRI image to detect Alzheimer's disease."
)

# Launch the interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8d2e4c271e65ce6b96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
